In [1]:
#Site CES 2025
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Configurer Selenium avec WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Charger la page principale
url = "https://exhibitors.ces.tech/8_0/#/searchtype/country/search/France/show/all"
driver.get(url)

# Attendre que la première liste de cartes charge
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.js-Card'))
)

# Affichez un message pour signaler que vous pouvez scroller
print("Défilez manuellement dans le navigateur pour charger toutes les entreprises.")
print("Scrollez jusqu'à ce que plus aucune entreprise ne se charge, puis revenez ici.")
input("Appuyez sur Entrée lorsque vous avez terminé le défilement.")

# Extraire les informations des cartes après défilement manuel
cards = driver.find_elements(By.CSS_SELECTOR, 'li.js-Card')
print(f"Total d'entreprises affichées : {len(cards)}")

# Liste pour stocker les entreprises
exposants = []

for index, card in enumerate(cards):
    try:
        # Récupérer le titre
        try:
            title_element = card.find_element(By.CSS_SELECTOR, 'h3.card-Title')
            name = title_element.text
        except:
            title_element = card.find_element(By.CSS_SELECTOR, 'a')
            name = title_element.text

        # Récupérer le lien
        try:
            link_element = card.find_element(By.CSS_SELECTOR, 'a')  # Modifier ici si nécessaire
            link = link_element.get_attribute('href')
        except Exception as e:
            link = None
            print(f"Erreur lors de l'extraction du lien pour {name}: {e}")

        exposants.append({'Nom': name, 'Lien': link})

    except Exception as e:
        print(f"Erreur inattendue pour la carte {index + 1} : {e}")
        continue

# Vérifiez les entreprises sans lien
print(f"Entreprises sans lien : {[e for e in exposants if e['Lien'] is None]}")

# Extraire les détails pour chaque exposant
for exposant in exposants:
    try:
        if exposant['Lien']:
            # Naviguer vers la page entreprise
            driver.get(exposant['Lien'])

            # Attendre que la page charge les détails
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h2.color-brand-02'))
            )

            # Description
            try:
                description = driver.find_element(By.CSS_SELECTOR, 'p.js-read-more.animated').text
                exposant['Description'] = description if description else "Non disponible"
            except Exception as e:
                exposant['Description'] = "Non disponible"
                print(f"Erreur lors de l'extraction de la description pour {exposant['Nom']}: {e}")

            # Catégories de produits
            try:
                categories = driver.find_elements(By.CSS_SELECTOR, 'div.grid.grid-3-col.grid__centered a')
                exposant['Catégories'] = [cat.text for cat in categories] if categories else []
            except Exception as e:
                exposant['Catégories'] = []
                print(f"Erreur lors de l'extraction des catégories pour {exposant['Nom']}: {e}")

            # Site web
            try:
                website = driver.find_element(By.CSS_SELECTOR, 'ul.showcase-web-phone a').get_attribute('href')
                exposant['Website'] = website if website else "Non disponible"
            except Exception as e:
                exposant['Website'] = "Non disponible"
                print(f"Erreur lors de l'extraction du site web pour {exposant['Nom']}: {e}")

            # LinkedIn
            try:
                linkedin = driver.find_element(By.CSS_SELECTOR, 'div.showcase-social.center.tc a').get_attribute('href')
                exposant['LinkedIn'] = linkedin if linkedin else "Non disponible"
            except Exception as e:
                exposant['LinkedIn'] = "Non disponible"
                print(f"Erreur lors de l'extraction du LinkedIn pour {exposant['Nom']}: {e}")

            # Adresse
            try:
                address = driver.find_element(By.CSS_SELECTOR, 'p.showcase-address.tc').text
                exposant['Adresse'] = address if address else "Non disponible"
            except Exception as e:
                exposant['Adresse'] = "Non disponible"
                print(f"Erreur lors de l'extraction de l'adresse pour {exposant['Nom']}: {e}")
        else:
            print(f"Pas de lien trouvé pour {exposant['Nom']}.")

    except Exception as e:
        print(f"Erreur lors de l'extraction des détails pour {exposant['Nom']} : {e}")

# Fermer le navigateur
driver.quit()

# Afficher les résultats
print(f"Nombre d'entreprises extraites : {len(exposants)}")
for exposant in exposants[:10]:  # Afficher les 10 premières entreprises pour vérification
    print(exposant)


Défilez manuellement dans le navigateur pour charger toutes les entreprises.
Scrollez jusqu'à ce que plus aucune entreprise ne se charge, puis revenez ici.


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x003BFD53+23747]
	(No symbol) [0x00347D54]
	(No symbol) [0x0021BE53]
	(No symbol) [0x001FD91B]
	(No symbol) [0x00287EFF]
	(No symbol) [0x0029AD49]
	(No symbol) [0x00281B96]
	(No symbol) [0x00253F3C]
	(No symbol) [0x00254EBD]
	GetHandleVerifier [0x0069AC73+3017699]
	GetHandleVerifier [0x006AB93B+3086507]
	GetHandleVerifier [0x006A40F2+3055714]
	GetHandleVerifier [0x00455AF0+637536]
	(No symbol) [0x00350A5D]
	(No symbol) [0x0034DA28]
	(No symbol) [0x0034DBC5]
	(No symbol) [0x003407F0]
	BaseThreadInitThunk [0x76E75D49+25]
	RtlInitializeExceptionChain [0x7748CEBB+107]
	RtlGetAppContainerNamedObjectPath [0x7748CE41+561]


In [ ]:
exposants

In [ ]:
import csv
# Exporter les données en CSV
csv_filename = "exposants.csv"
csv_columns = ['Nom', 'Lien', 'Description', 'Catégories', 'Website', 'LinkedIn', 'Adresse']

try:
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for exposant in exposants:
            # Convertir la liste des catégories en chaîne séparée par des virgules
            exposant['Catégories'] = ", ".join(exposant.get('Catégories', []))
            writer.writerow(exposant)
    print(f"Les données ont été exportées avec succès dans le fichier '{csv_filename}'.")
except IOError as e:
    print(f"Erreur lors de l'exportation en CSV : {e}")

In [ ]:
#Site CES 2025 - AI World
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Configurer Selenium avec WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Charger la page principale
url = "https://exhibitors.ces.tech/8_0/#/searchtype/category/search/191/show/all"
driver.get(url)

# Attendre que la première liste de cartes charge
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.js-Card'))
)

# Affichez un message pour signaler que vous pouvez scroller
print("Défilez manuellement dans le navigateur pour charger toutes les entreprises.")
print("Scrollez jusqu'à ce que plus aucune entreprise ne se charge, puis revenez ici.")
input("Appuyez sur Entrée lorsque vous avez terminé le défilement.")

# Extraire les informations des cartes après défilement manuel
cards = driver.find_elements(By.CSS_SELECTOR, 'li.js-Card')
print(f"Total d'entreprises affichées : {len(cards)}")

# Liste pour stocker les entreprises
exposants = []

for index, card in enumerate(cards):
    try:
        # Récupérer le titre
        try:
            title_element = card.find_element(By.CSS_SELECTOR, 'h3.card-Title')
            name = title_element.text
        except:
            title_element = card.find_element(By.CSS_SELECTOR, 'a')
            name = title_element.text

        # Récupérer le lien
        try:
            link_element = card.find_element(By.CSS_SELECTOR, 'a')  # Modifier ici si nécessaire
            link = link_element.get_attribute('href')
        except Exception as e:
            link = None
            print(f"Erreur lors de l'extraction du lien pour {name}: {e}")

        exposants.append({'Nom': name, 'Lien': link})

    except Exception as e:
        print(f"Erreur inattendue pour la carte {index + 1} : {e}")
        continue

# Vérifiez les entreprises sans lien
print(f"Entreprises sans lien : {[e for e in exposants if e['Lien'] is None]}")

# Extraire les détails pour chaque exposant
for exposant in exposants:
    try:
        if exposant['Lien']:
            # Naviguer vers la page entreprise
            driver.get(exposant['Lien'])

            # Attendre que la page charge les détails
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h2.color-brand-02'))
            )

            # Description
            try:
                description = driver.find_element(By.CSS_SELECTOR, 'p.js-read-more.animated').text
                exposant['Description'] = description if description else "Non disponible"
            except Exception as e:
                exposant['Description'] = "Non disponible"
                print(f"Erreur lors de l'extraction de la description pour {exposant['Nom']}: {e}")

            # Catégories de produits
            try:
                categories = driver.find_elements(By.CSS_SELECTOR, 'div.grid.grid-3-col.grid__centered a')
                exposant['Catégories'] = [cat.text for cat in categories] if categories else []
            except Exception as e:
                exposant['Catégories'] = []
                print(f"Erreur lors de l'extraction des catégories pour {exposant['Nom']}: {e}")

            # Site web
            try:
                website = driver.find_element(By.CSS_SELECTOR, 'ul.showcase-web-phone a').get_attribute('href')
                exposant['Website'] = website if website else "Non disponible"
            except Exception as e:
                exposant['Website'] = "Non disponible"
                print(f"Erreur lors de l'extraction du site web pour {exposant['Nom']}: {e}")

            # LinkedIn
            try:
                linkedin = driver.find_element(By.CSS_SELECTOR, 'div.showcase-social.center.tc a').get_attribute('href')
                exposant['LinkedIn'] = linkedin if linkedin else "Non disponible"
            except Exception as e:
                exposant['LinkedIn'] = "Non disponible"
                print(f"Erreur lors de l'extraction du LinkedIn pour {exposant['Nom']}: {e}")

            # Adresse
            try:
                address = driver.find_element(By.CSS_SELECTOR, 'p.showcase-address.tc').text
                exposant['Adresse'] = address if address else "Non disponible"
            except Exception as e:
                exposant['Adresse'] = "Non disponible"
                print(f"Erreur lors de l'extraction de l'adresse pour {exposant['Nom']}: {e}")
        else:
            print(f"Pas de lien trouvé pour {exposant['Nom']}.")

    except Exception as e:
        print(f"Erreur lors de l'extraction des détails pour {exposant['Nom']} : {e}")

# Fermer le navigateur
driver.quit()

# Afficher les résultats
print(f"Nombre d'entreprises extraites : {len(exposants)}")
for exposant in exposants[:10]:  # Afficher les 10 premières entreprises pour vérification
    print(exposant)


In [ ]:
import csv
# Exporter les données en CSV
csv_filename = "exposants.csv"
csv_columns = ['Nom', 'Lien', 'Description', 'Catégories', 'Website', 'LinkedIn', 'Adresse']

try:
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for exposant in exposants:
            # Convertir la liste des catégories en chaîne séparée par des virgules
            exposant['Catégories'] = ", ".join(exposant.get('Catégories', []))
            writer.writerow(exposant)
    print(f"Les données ont été exportées avec succès dans le fichier '{csv_filename}'.")
except IOError as e:
    print(f"Erreur lors de l'exportation en CSV : {e}")

In [ ]:
#Les pepites tech
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

# Configurer Selenium avec WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Charger la page principale
url = "https://lespepitestech.com/best-of-pepites"
driver.get(url)

# Attendre que la première page charge
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.s-e-content'))
)

# Afficher un message pour naviguer manuellement à la bonne page de départ
input("Naviguez manuellement vers la page de départ, puis appuyez sur Entrée pour commencer le scraping des pages suivantes.")

# Liste pour stocker les entreprises
entreprises = []

# Fonction pour scraper les données d'une page
def scraper_page():
    cards = driver.find_elements(By.CSS_SELECTOR, 'div.s-e-content')
    for card in cards:
        try:
            entreprise = {}

            # Nom de l'entreprise
            try:
                nom = card.find_element(By.CSS_SELECTOR, 'div.s-e-title h3').text
                entreprise['Nom'] = nom
            except Exception:
                entreprise['Nom'] = "Non disponible"

            # Description de l'entreprise
            try:
                description = card.find_element(By.CSS_SELECTOR, 'div.s-u-summary').text
                entreprise['Description'] = description
            except Exception:
                entreprise['Description'] = "Non disponible"

            # Catégories
            try:
                categories = card.find_elements(By.CSS_SELECTOR, 'div.lpt-dropdown-category span.tag a')
                entreprise['Catégories'] = [cat.text for cat in categories]
            except Exception:
                entreprise['Catégories'] = []

            # Votes
            try:
                votes = card.find_element(By.CSS_SELECTOR, 'div.counter.card-vote .alternate-votes-display').text.strip()
                entreprise['Votes'] = votes
            except Exception:
                entreprise['Votes'] = "Non disponible"

            # Commentaires
            try:
                commentaires = card.find_element(By.CSS_SELECTOR, 'div.counter.card-comment').text.strip()
                entreprise['Commentaires'] = commentaires
            except Exception:
                entreprise['Commentaires'] = "Non disponible"

            # Site Web
            try:
                site_web = card.find_element(By.CSS_SELECTOR, 'div.s-e-link a').get_attribute('href')
                entreprise['Site Web'] = site_web
            except Exception:
                entreprise['Site Web'] = "Non disponible"

            # Localisation
            try:
                localisation = card.find_element(By.CSS_SELECTOR, 'div.card-location a').get_attribute('data-original-title')
                entreprise['Localisation'] = localisation
            except Exception:
                entreprise['Localisation'] = "Non disponible"

            entreprises.append(entreprise)

        except Exception as e:
            print(f"Erreur lors de l'extraction des données pour une entreprise : {e}")
            continue

# Scraper les données des pages automatiquement
while True:
    scraper_page()
    try:
        # Trouver le bouton "Page suivante" via le titre ou le texte
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[title="Aller à la page suivante"]')
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)  # Attendre que la nouvelle page charge
    except Exception:
        print("Aucune autre page trouvée. Fin du scraping.")
        break

# Fermer le navigateur
driver.quit()

# Exporter les données en CSV
csv_filename = "pepites_scraped.csv"
csv_columns = ['Nom', 'Description', 'Catégories', 'Votes', 'Commentaires', 'Site Web', 'Localisation']

try:
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for entreprise in entreprises:
            writer.writerow(entreprise)
    print(f"Les données ont été exportées avec succès dans le fichier '{csv_filename}'.")
except IOError as e:
    print(f"Erreur lors de l'exportation en CSV : {e}")

# Afficher les premières entreprises
for entreprise in entreprises[:10]:
    print(entreprise)


In [1]:
#lafrenchtech.com
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

# Configurer Selenium avec WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Charger la page principale
url = "https://ecosystem.lafrenchtech.com/companies/f/data_type/anyof_Verified/founding_or_hq_slug_locations/anyof_france?selectedColumns=name%2CstartupRankingRating%2CcompanyMarket%2Ctype%2ClaunchDate%2Cvaluation%2CtotalFunding%2ChqLocations%2ClastFundingEnhanced%2Crevenue%2CcompanyStatus%2CgrowthStage%2CinnovationCorporateRank%2CkpiMarketFirm"
driver.get(url)

# Attendre que la première page charge
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.s-e-content'))
)

# Afficher un message pour naviguer manuellement à la bonne page de départ
input("Naviguez manuellement vers la page de départ, puis appuyez sur Entrée pour commencer le scraping des pages suivantes.")

# Liste pour stocker les entreprises
entreprises = []

# Fonction pour scraper les données d'une page
def scraper_page():
    cards = driver.find_elements(By.CSS_SELECTOR, 'div.s-e-content')
    for card in cards:
        try:
            entreprise = {}

            # Nom de l'entreprise
            try:
                nom = card.find_element(By.CSS_SELECTOR, 'div.s-e-title h3').text
                entreprise['Nom'] = nom
            except Exception:
                entreprise['Nom'] = "Non disponible"

            # Description de l'entreprise
            try:
                description = card.find_element(By.CSS_SELECTOR, 'div.s-u-summary').text
                entreprise['Description'] = description
            except Exception:
                entreprise['Description'] = "Non disponible"

            # Catégories
            try:
                categories = card.find_elements(By.CSS_SELECTOR, 'div.lpt-dropdown-category span.tag a')
                entreprise['Catégories'] = [cat.text for cat in categories]
            except Exception:
                entreprise['Catégories'] = []

            # Votes
            try:
                votes = card.find_element(By.CSS_SELECTOR, 'div.counter.card-vote .alternate-votes-display').text.strip()
                entreprise['Votes'] = votes
            except Exception:
                entreprise['Votes'] = "Non disponible"

            # Commentaires
            try:
                commentaires = card.find_element(By.CSS_SELECTOR, 'div.counter.card-comment').text.strip()
                entreprise['Commentaires'] = commentaires
            except Exception:
                entreprise['Commentaires'] = "Non disponible"

            # Site Web
            try:
                site_web = card.find_element(By.CSS_SELECTOR, 'div.s-e-link a').get_attribute('href')
                entreprise['Site Web'] = site_web
            except Exception:
                entreprise['Site Web'] = "Non disponible"

            # Localisation
            try:
                localisation = card.find_element(By.CSS_SELECTOR, 'div.card-location a').get_attribute('data-original-title')
                entreprise['Localisation'] = localisation
            except Exception:
                entreprise['Localisation'] = "Non disponible"

            entreprises.append(entreprise)

        except Exception as e:
            print(f"Erreur lors de l'extraction des données pour une entreprise : {e}")
            continue

# Scraper les données des pages automatiquement
while True:
    scraper_page()
    try:
        # Trouver le bouton "Page suivante" via le titre ou le texte
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[title="Aller à la page suivante"]')
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)  # Attendre que la nouvelle page charge
    except Exception:
        print("Aucune autre page trouvée. Fin du scraping.")
        break

# Fermer le navigateur
driver.quit()

# Exporter les données en CSV
csv_filename = "pepites_scraped.csv"
csv_columns = ['Nom', 'Description', 'Catégories', 'Votes', 'Commentaires', 'Site Web', 'Localisation']

try:
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for entreprise in entreprises:
            writer.writerow(entreprise)
    print(f"Les données ont été exportées avec succès dans le fichier '{csv_filename}'.")
except IOError as e:
    print(f"Erreur lors de l'exportation en CSV : {e}")

# Afficher les premières entreprises
for entreprise in entreprises[:10]:
    print(entreprise)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.265)
Stacktrace:
	GetHandleVerifier [0x0100FD53+23747]
	(No symbol) [0x00F97D54]
	(No symbol) [0x00E6BE53]
	(No symbol) [0x00E4D91B]
	(No symbol) [0x00ED7EFF]
	(No symbol) [0x00EEAD49]
	(No symbol) [0x00ED1B96]
	(No symbol) [0x00EA3F3C]
	(No symbol) [0x00EA4EBD]
	GetHandleVerifier [0x012EAC73+3017699]
	GetHandleVerifier [0x012FB93B+3086507]
	GetHandleVerifier [0x012F40F2+3055714]
	GetHandleVerifier [0x010A5AF0+637536]
	(No symbol) [0x00FA0A5D]
	(No symbol) [0x00F9DA28]
	(No symbol) [0x00F9DBC5]
	(No symbol) [0x00F907F0]
	BaseThreadInitThunk [0x75F05D49+25]
	RtlInitializeExceptionChain [0x77BDCEBB+107]
	RtlGetAppContainerNamedObjectPath [0x77BDCE41+561]


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
import time

# Configuration du WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--ignore-certificate-errors")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Fonction pour accepter automatiquement les cookies
def accept_cookies(driver):
    try:
        # Attendre que le bouton "J'accepte" apparaisse
        wait = WebDriverWait(driver, 10)
        cookie_button = wait.until(
            EC.element_to_be_clickable((By.ID, "axeptio_btn_acceptAll"))
        )
        cookie_button.click()
        print("Cookies acceptés.")
        time.sleep(2)  # Pause après avoir cliqué
    except Exception as e:
        print("Aucun bouton cookies détecté ou erreur :", e)

# Fonction pour extraire les données d'une entreprise
def scrape_company_data(driver, company_url):
    driver.get(company_url)
    time.sleep(2)  # Pause pour le chargement

    try:
        name = driver.find_element(By.CSS_SELECTOR, "h1.incubator-top__title").text
    except:
        name = None

    try:
        description = driver.find_element(By.CSS_SELECTOR, "div.article-text.article-text--no-initial").text
    except:
        description = None

    try:
        headquarters = driver.find_element(By.XPATH, "//div[text()='Siège social']/following-sibling::div").text
    except:
        headquarters = None

    try:
        creation_date = driver.find_element(By.XPATH, "//div[text()='Date de création']/following-sibling::div").text
    except:
        creation_date = None

    try:
        website = driver.find_element(By.CSS_SELECTOR, "a.incubator__aside-link").get_attribute("href")
    except:
        website = None

    try:
        logo = driver.find_element(By.CSS_SELECTOR, "img.incubator__aside-logo__img").get_attribute("src")
    except:
        logo = None

    try:
        last_funding_date = driver.find_element(By.CSS_SELECTOR, "span.financement-card__date").text
    except:
        last_funding_date = None

    try:
        funding_series = driver.find_element(By.CSS_SELECTOR, "span.financement-card__serie").text
    except:
        funding_series = None

    try:
        funding_amount = driver.find_element(By.CSS_SELECTOR, "div.financement-card__amount").text
    except:
        funding_amount = None

    try:
        hashtags = [tag.text for tag in driver.find_elements(By.CSS_SELECTOR, "span.incubator-top__hashtag")]
    except:
        hashtags = []

    return {
        "Nom": name,
        "Description": description,
        "Siège": headquarters,
        "Date de création": creation_date,
        "Site internet": website,
        "Logo": logo,
        "Date dernier financement": last_funding_date,
        "Série": funding_series,
        "Montant": funding_amount,
        "Hashtags": hashtags
    }

try:
    # Charger la page principale
    url = "https://www.maddyness.com/entreprise/"
    driver.get(url)
    time.sleep(5)

    # Accepter les cookies
    accept_cookies(driver)

    print("Veuillez charger toutes les pages manuellement. Appuyez sur Entrée lorsque vous avez terminé.")
    input()  # Attendre que l'utilisateur appuie sur Entrée

    # Récupérer tous les liens des entreprises
    links = [a.get_attribute("href") for a in driver.find_elements(By.CSS_SELECTOR, "a.enterprise__item")]
    print(f"{len(links)} liens trouvés.")

    # Extraire les données de chaque entreprise
    all_data = []
    for link in links:
        print(f"Extraction des données de {link}...")
        data = scrape_company_data(driver, link)
        all_data.append(data)

    # Sauvegarder les données dans un fichier JSON
    with open("entreprises_data.json", "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)

    print(f"Données de {len(all_data)} entreprises sauvegardées dans 'entreprises_data.json'.")

except Exception as e:
    print(f"Erreur : {e}")

finally:
    driver.quit()



Aucun bouton cookies détecté ou erreur : Message: 
Stacktrace:
	GetHandleVerifier [0x0100FD53+23747]
	(No symbol) [0x00F97D54]
	(No symbol) [0x00E6BE53]
	(No symbol) [0x00EAFCA6]
	(No symbol) [0x00EAFEEB]
	(No symbol) [0x00EED852]
	(No symbol) [0x00ED1E44]
	(No symbol) [0x00EEB41E]
	(No symbol) [0x00ED1B96]
	(No symbol) [0x00EA3F3C]
	(No symbol) [0x00EA4EBD]
	GetHandleVerifier [0x012EAC73+3017699]
	GetHandleVerifier [0x012FB93B+3086507]
	GetHandleVerifier [0x012F40F2+3055714]
	GetHandleVerifier [0x010A5AF0+637536]
	(No symbol) [0x00FA0A5D]
	(No symbol) [0x00F9DA28]
	(No symbol) [0x00F9DBC5]
	(No symbol) [0x00F907F0]
	BaseThreadInitThunk [0x75F05D49+25]
	RtlInitializeExceptionChain [0x77BDCEBB+107]
	RtlGetAppContainerNamedObjectPath [0x77BDCE41+561]

Veuillez charger toutes les pages manuellement. Appuyez sur Entrée lorsque vous avez terminé.
2256 liens trouvés.
Extraction des données de https://www.maddyness.com/entreprise/lendix/...
Extraction des données de https://www.maddyness.com/

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
import time

# Configuration Selenium
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")
options.add_argument("--disable-gpu")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)

# Supprimez le mode headless pour afficher le navigateur
# options.add_argument("--headless")  # Enlevez cette ligne pour voir le navigateur

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

try:
    base_url = "https://lehub.web.bpifrance.fr/search"
    driver.delete_all_cookies()  # Efface les cookies avant de charger la page
    driver.get(base_url)
    print("Page principale chargée en mode normal...")

    # Interagir avec les cookies
    try:
        accept_cookies = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "axeptio_btn_acceptAll"))
        )
        accept_cookies.click()
        print("Cookies acceptés.")
    except Exception:
        print("Aucun bouton de cookies détecté.")

    # Vérifier et cliquer sur le bouton "Voir tout"
    try:
        voir_tout = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "Voir tout"))
        )
        voir_tout.click()
        print("Bouton 'Voir tout' cliqué.")
    except Exception as e:
        print("Le bouton 'Voir tout' n'a pas été trouvé ou cliqué :", e)

    # Liste pour stocker les données
    startups_data = []
    page = 1

    while True:
        print(f"Chargement de la page {page}...")
        # Attendre que les startups apparaissent
        try:
            WebDriverWait(driver, 30).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href^='/startup/']"))
            )
        except Exception as e:
            print(f"Le contenu n'a pas chargé correctement sur la page {page}: {e}")
            break

        # Récupérer les liens des startups
        startups = driver.find_elements(By.CSS_SELECTOR, "a[href^='/startup/']")
        if not startups:
            print(f"Aucun contenu trouvé sur la page {page}. Arrêt.")
            break

        for startup in startups:
            startups_data.append(startup.get_attribute("href"))

        print(f"Éléments trouvés sur la page {page} : {len(startups)}")
        print(f"Total des startups collectées : {len(startups_data)}")

        # Capture d'écran pour debugging
        driver.save_screenshot(f"debug_page_{page}.png")

        # Passer à la page suivante
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.ais-Pagination-item--nextPage a"))
            )
            next_button.click()
            page += 1
            time.sleep(2)  # Pause pour éviter les problèmes de chargement
        except Exception:
            print("Fin de la pagination atteinte.")
            break

    # Sauvegarder les données
    with open("bpifrance_startups_links.json", "w", encoding="utf-8") as f:
        json.dump(startups_data, f, ensure_ascii=False, indent=4)
    print(f"Scrapping terminé. Total: {len(startups_data)} startups sauvegardées dans 'bpifrance_startups_links.json'.")

except Exception as e:
    print(f"Erreur : {e}")

finally:
    driver.quit()


Page principale chargée en mode normal...
Aucun bouton de cookies détecté.
Le bouton 'Voir tout' n'a pas été trouvé ou cliqué : Message: 
Stacktrace:
	GetHandleVerifier [0x00C0FD53+23747]
	(No symbol) [0x00B97D54]
	(No symbol) [0x00A6BE53]
	(No symbol) [0x00AAFCA6]
	(No symbol) [0x00AAFEEB]
	(No symbol) [0x00AED852]
	(No symbol) [0x00AD1E44]
	(No symbol) [0x00AEB41E]
	(No symbol) [0x00AD1B96]
	(No symbol) [0x00AA3F3C]
	(No symbol) [0x00AA4EBD]
	GetHandleVerifier [0x00EEAC73+3017699]
	GetHandleVerifier [0x00EFB93B+3086507]
	GetHandleVerifier [0x00EF40F2+3055714]
	GetHandleVerifier [0x00CA5AF0+637536]
	(No symbol) [0x00BA0A5D]
	(No symbol) [0x00B9DA28]
	(No symbol) [0x00B9DBC5]
	(No symbol) [0x00B907F0]
	BaseThreadInitThunk [0x75F05D49+25]
	RtlInitializeExceptionChain [0x77BDCEBB+107]
	RtlGetAppContainerNamedObjectPath [0x77BDCE41+561]

Chargement de la page 1...
Éléments trouvés sur la page 1 : 18
Total des startups collectées : 18
Chargement de la page 2...
Éléments trouvés sur la page